In [40]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import os
#import skmultilearn 

os.getcwd()
# Ml  packages
from sklearn.preprocessing import StandardScaler
from nltk.tokenize import TreebankWordTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# Split Dataset into training and text
from sklearn.model_selection import train_test_split
#feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer
# text preprocessing
import string
import nltk
from langdetect import detect
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest ,chi2
from textblob import TextBlob
from textblob import TextBlob
from manual_spellchecker import spell_checker
from spellchecker import SpellChecker
import re
import nltk
import spacy
import string

In [264]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv("test_with_no_labels.csv")

In [265]:
df["message"] = df["message"].str.lower()
df.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesn't think carbon di...,625221
1,1,it's not like we lack evidence of anthropogeni...,126103
2,2,rt @rawstory: researchers say we have three ye...,698562
3,1,#todayinmaker# wired : 2016 was a pivotal year...,573736
4,1,"rt @soynoviodetodas: it's 2016, and a racist, ...",466954


In [266]:

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["message"] = df["message"].apply(lambda text: remove_punctuation(text))
df.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,its not like we lack evidence of anthropogenic...,126103
2,2,rt rawstory researchers say we have three year...,698562
3,1,todayinmaker wired 2016 was a pivotal year in...,573736
4,1,rt soynoviodetodas its 2016 and a racist sexis...,466954


In [267]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["message"] = df["message"].apply(lambda text: remove_stopwords(text))
df.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,like lack evidence anthropogenic global warming,126103
2,2,rt rawstory researchers say three years act cl...,698562
3,1,todayinmaker wired 2016 pivotal year war clima...,573736
4,1,rt soynoviodetodas 2016 racist sexist climate ...,466954


In [268]:
from collections import Counter
cnt = Counter()
for text in df["message"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('climate', 12897),
 ('change', 12212),
 ('rt', 9722),
 ('global', 3764),
 ('warming', 3413),
 ('trump', 1935),
 ('believe', 1154),
 ('us', 894),
 ('amp', 872),
 ('doesnt', 812)]

In [269]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(2)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["message"] = df["message"].apply(lambda text: remove_freqwords(text))
df.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,like lack evidence anthropogenic global warming,126103
2,2,rt rawstory researchers say three years act it...,698562
3,1,todayinmaker wired 2016 pivotal year war https...,573736
4,1,rt soynoviodetodas 2016 racist sexist denying ...,466954


In [270]:
n_rare_words = 2
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["message"] = df["message"].apply(lambda text: remove_rarewords(text))
df.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,like lack evidence anthropogenic global warming,126103
2,2,rt rawstory researchers say three years act it...,698562
3,1,todayinmaker wired 2016 pivotal year war https...,573736
4,1,rt soynoviodetodas 2016 racist sexist denying ...,466954


In [271]:
from nltk.stem.porter import PorterStemmer

 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

df["message"] = df["message"].apply(lambda text: stem_words(text))
df.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,like lack evid anthropogen global warm,126103
2,2,rt rawstori research say three year act it’ la...,698562
3,1,todayinmak wire 2016 pivot year war httpstco44...,573736
4,1,rt soynoviodetoda 2016 racist sexist deni bigo...,466954


In [272]:
from nltk.stem.snowball import SnowballStemmer
#SnowballStemmer.languages

In [273]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["message"] = df["message"].apply(lambda text: lemmatize_words(text))
df.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dio...,625221
1,1,like lack evid anthropogen global warm,126103
2,2,rt rawstori research say three year act it’ la...,698562
3,1,todayinmak wire 2016 pivot year war httpstco44...,573736
4,1,rt soynoviodetoda 2016 racist sexist deni bigo...,466954


In [274]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)



In [275]:
df['message'] = df['message'].apply(remove_emoji)

In [276]:
df['message'] = df['message'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [277]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [278]:
df['message'] = df['message'].apply(remove_urls)

In [279]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)


In [280]:
df['message'] = df['message'].apply(remove_html)

In [281]:
from bs4 import BeautifulSoup

def remove_htmls(text):
    return BeautifulSoup(text, "lxml").text

In [282]:
df['message'] = df['message'].apply(remove_htmls)

In [74]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [75]:
#df['message'] = df['message'].apply(correct_spellings)

KeyboardInterrupt: 

In [283]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['message'], df['sentiment'], test_size = 0.2, random_state = 42)

In [284]:
from sklearn.svm import LinearSVC
lsvc_ = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lsvc', LinearSVC())
])

In [285]:
lsvc_params = {
    'lsvc__C': [0.2, 0.3],
    'tfidf__ngram_range' : [(1,1),(1,2)],
    'lsvc__max_iter': [500, 1000],
    'tfidf__min_df': [2, 1],
    'tfidf__max_df': [0.7, 0.8], 
    'lsvc__class_weight': ['balanced']
}

In [286]:
ls_cv = GridSearchCV(
    lsvc_, 
    param_grid = lsvc_params, 
    cv = 5, 
    scoring = 'f1_weighted'
)

In [287]:
ls_cv.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('lsvc', LinearSVC())]),
             param_grid={'lsvc__C': [0.2, 0.3],
                         'lsvc__class_weight': ['balanced'],
                         'lsvc__max_iter': [500, 1000],
                         'tfidf__max_df': [0.7, 0.8], 'tfidf__min_df': [2, 1],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_weighted')

In [288]:
ls_cv.best_params_

{'lsvc__C': 0.3,
 'lsvc__class_weight': 'balanced',
 'lsvc__max_iter': 500,
 'tfidf__max_df': 0.7,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 2)}

In [289]:
ls_cv.best_score_

0.7227434894674502

In [290]:
ls_ypred = ls_cv.predict(x_test)

In [291]:
print(classification_report(y_test, ls_ypred))

              precision    recall  f1-score   support

          -1       0.66      0.55      0.60       278
           0       0.57      0.44      0.50       425
           1       0.80      0.84      0.82      1755
           2       0.72      0.78      0.75       706

    accuracy                           0.75      3164
   macro avg       0.69      0.65      0.67      3164
weighted avg       0.74      0.75      0.74      3164



In [293]:
lr_ = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('lr', LogisticRegression())
])

In [294]:
lr_params = {
    'lr__C': [0.2, 0.3],
    'lr__class_weight': ['balanced'],
    'tfidf__max_df': [0.8, 0.7], 
    'tfidf__ngram_range' : [(1,1),(1,2)], 
    'tfidf__min_df': [2, 1]
}

In [295]:
lr_cv = GridSearchCV(
    lr_,
    param_grid = lr_params,
    scoring = 'f1_weighted',
    cv = 5
)

In [296]:
lr_cv.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('lr', LogisticRegression())]),
             param_grid={'lr__C': [0.2, 0.3], 'lr__class_weight': ['balanced'],
                         'tfidf__max_df': [0.8, 0.7], 'tfidf__min_df': [2, 1],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             scoring='f1_weighted')

In [297]:
lr_cv.best_params_

{'lr__C': 0.3,
 'lr__class_weight': 'balanced',
 'tfidf__max_df': 0.8,
 'tfidf__min_df': 2,
 'tfidf__ngram_range': (1, 2)}

In [298]:
lr_cv.best_score_

0.6723070037939427

In [299]:
lr_ypred = lr_cv.predict(X_test)

NameError: name 'X_test' is not defined

In [ ]:
print(classification_report(y_test, lr_ypred))

## **testing**

In [248]:
df_test =  pd.read_csv("test_with_no_labels.csv")

In [249]:
df_test['processed_message'] = df_test['message'].apply(remove_punctuation)

In [250]:
df_test['processed_message'] = df_test['processed_message'].apply(remove_stopwords)

In [251]:
df_test['processed_message'] = df_test['processed_message'].apply(remove_emoji)

In [252]:
df_test['processed_message'] = df_test['processed_message'].apply(remove_urls)

In [253]:
df_test['processed_message'] = df_test['processed_message'].apply(remove_html)

In [254]:
df_test['processed_message'] = df_test['processed_message'].apply(stop_words)

In [255]:
df_test['processed_message'] = df_test['processed_message'].apply(text_cleaning)

In [256]:
df_test['processed_message'] = df_test['processed_message'].apply(remove_htmls)

In [257]:
df_test['processed_message'] = df_test['processed_message'].apply(lambda text: remove_freqwords(text))

In [258]:
df_test['processed_message'] = df_test['processed_message'].apply(lambda text: remove_rarewords(text))

In [259]:
df_test['processed_message'] = df_test['processed_message'].apply(lambda text: lemmatize_words(text))

In [260]:
df_test.head()

,message,tweetid,processed_message
0,Europe will now be looking to China to make su...,169760,europe looking cna make sure alone fighting ch...
1,Combine this with the polling of staffers re c...,35326,combine polling staffer woman right fascist st...
2,"The scary, unimpeachable evidence that climate...",224985,the scary unimpeachable evidence already https...
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,karoli morgfair osborneink dailykos putin got ...
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,fakewillmoore female orgasm cause global sarca...


In [261]:
predictions = ls_cv.predict(df_test['processed_message'])

In [262]:
sub = pd.DataFrame({'tweetid':df_test['tweetid'],'sentiment':predictions})
sub.to_csv('Climate_change_pred.csv',index=False)

In [263]:
sub.head()

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,2
